In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob, os, random
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from scipy.signal import savgol_filter
from scipy import interpolate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
np.random.seed( 0 )
sns.set_style( "darkgrid" )


In [ ]:
df = pd.read_parquet("/Users/lselig/selig-sandbox/finance/.data/fundamental_analysis_neglog.parquet")
df = df[(df.period == "Q2") & (df.calendarYear == "2022")]
tickers = df.symbol.values
periods = df.period.values
calendarYears = df.calendarYear.values
qualitative_data = np.array((tickers, periods, calendarYears)).T
df = df.drop(columns = ["period", "calendarYear", "symbol", "acceptedDate"])

In [ ]:
features = np.array(df)
scaler = StandardScaler()
features =  scaler.fit_transform(features)
features = np.clip(features, -5, 5)
print(features.shape)
%matplotlib widget 
# for f in features:
#     print(np.nanmin(f), np.nanmax(f))
#     sns.kdeplot(f)
# plt.title("Distribution of final features")


In [ ]:
pca = PCA(n_components=2)
pca.fit(features.T)
# print(pca.explained_variance_ratio_)
# print(np.sum(pca.explained_variance_ratio_))
# print(pca.singular_values_)
# print(components)
# plt.hist2d(components[:, 0], components[:, 1], bins = 50)
components = pca.components_.T
print(components.shape)
# components[0,:]
plt.close()
plt.hist2d(components[:, 0], components[:, 1], bins = 60)
plt.show()
# plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation

clst = KMeans(n_clusters=3, random_state=0).fit(components)
# clst = AffinityPropagation( random_state=0).fit(components)
labels = clst.labels_.reshape(-1, 1)
pca_and_labels = np.hstack((components, labels))
print(pca_and_labels.shape)
clst.labels_.shape
fig, axs = plt.subplots(1, 2, figsize = (12, 8))
for g in np.unique(labels):
    idxs = np.where(pca_and_labels[:, 2] == g)[0]
    cluster = pca_and_labels[idxs]
    for i, c in enumerate(cluster):
        print(c[0], c[1], qualitative_data[:, 0][idxs[i]])
        axs[0].text(c[0], c[1], s = qualitative_data[:, 0][idxs[i]])
    axs[0].scatter(cluster[:, 0],  cluster[:, 1], label = g)
axs[0].set_xlabel("PCA1")
axs[0].set_ylabel("PCA2")
axs[1].hist2d(pca_and_labels[:, 0], pca_and_labels[:, 1], bins = 40)
axs[1].set_xlabel("PCA1")
axs[1].set_ylabel("PCA2")
axs[0].legend()
plt.show()

    
# plt.scatter(pca_and_labels[:, 0], pca_and_labels[: ,1], group = )
# plt.close()
# plt.hist(kmeans.labels_)
# plt.show()


In [ ]:
# the tickers where PCA2 is an outlier are all related to Federal National Mortgage Association (FNMA)
# not sure if it can be exploited
interesting = np.argwhere(pca_and_labels[: ,1] >= 0.08)
interesting_tickers = tickers[interesting]
# print(interesting_tickers)

interesting = np.argwhere(pca_and_labels[: ,1] <= -0.0365)
interesting_tickers = tickers[interesting]
print(interesting_tickers)

In [ ]:

interesting = np.argwhere(pca_and_labels[: ,0] >= 0.0055)
interesting_tickers = qualitative_data[interesting]
print(interesting_tickers)